# Options IV Calculation Test

This notebook tests the options data fetching and implied volatility (IV) calculation pipeline.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from adapters.options_adapter import OptionsAdapter
from adapters.rates_adapter import RatesAdapter
from adapters.ticker_adapter import TickerAdapter
from models.options_data import OptionsRequest, OptionType
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from engines.IV_smile import IVEngine


In [3]:
from update_rates import updateRates
updateRates()

FRED_API_KEY present: yes
Starting updateRates...
Starting rate updates...
Updated discount factors: added 1 new records
No new SOFR data available from 2025-12-19
Rate updates completed.
updateRates completed.
Last discount_factors date: 2025-12-18
SOFR update was included.


## Setup

Initialize the adapter and create the options request.


In [4]:
adapter = OptionsAdapter()

# Simulate duration selection (e.g. '1y' from frontend)
today = date.today()
expiry_end = today + relativedelta(weeks=52)

req = OptionsRequest(
    ticker="MSFT",
    optionType=OptionType.CALL,
    expiryStart=today,
    expiryEnd=expiry_end,
    moneynessMin=0.8,
    moneynessMax=1.2,
)


## Fetch Options Data


In [5]:
print(f"Requesting options for {req.ticker} until {expiry_end}...")
df = adapter.fetch_option_chain(req)
print(f"Fetched {len(df)} contracts.")



Requesting options for MSFT until 2026-12-18...
Fetched 610 contracts.


## Display Options Data


In [6]:
df[["optionType", "strike", "timeToExpiry", "midPrice", "expiry"]]

,optionType,strike,timeToExpiry,midPrice,expiry
0,call,390.0,0.000010,95.995,2025-12-19
1,call,395.0,0.000010,90.675,2025-12-19
2,call,400.0,0.000010,84.375,2025-12-19
3,call,405.0,0.000010,80.925,2025-12-19
4,call,410.0,0.000010,75.200,2025-12-19
...,...,...,...,...,...
605,call,560.0,0.996972,28.915,2026-12-18
606,call,565.0,0.996972,27.715,2026-12-18
607,call,570.0,0.996972,25.995,2026-12-18
608,call,575.0,0.996972,25.040,2026-12-18


## Calculate Implied Volatility


In [7]:
from engines.zero_rates import ZeroRatesEngine

df["rate"] = ZeroRatesEngine.interpolate_zero_rate(df, tte_col="timeToExpiry")

base_info = TickerAdapter.fetchBasic(req.ticker)
div = base_info.dividendYield
spot = base_info.spot
#
#print(f"Spot price: ${spot:.2f}")
#print(f"Risk-free rate: {rate:.4f}")
#print(f"Dividend yield: {div:.4f}%")
#print("\nCalculating IVs...")
#
surface_data = IVEngine.generateIVSmile(df, df["rate"], div, spot, OptionType.CALL) #type: ignore
surface_data.dropna(inplace=True)




In [8]:
print("\nIV Calculation Results:")
surface_data


IV Calculation Results:


,type,K,T,Price,rate,expiry,iv,S
32,call,490.0,0.000010,0.010,0.037043,2025-12-19,0.604551,488.215
61,call,400.0,0.018890,85.120,0.037043,2025-12-26,1.099673,488.215
62,call,405.0,0.018890,80.645,0.037043,2025-12-26,1.088155,488.215
63,call,410.0,0.018890,74.740,0.037043,2025-12-26,0.956875,488.215
64,call,415.0,0.018890,69.950,0.037043,2025-12-26,0.920411,488.215
...,...,...,...,...,...,...,...,...
605,call,560.0,0.996972,28.915,0.034594,2026-12-18,0.917097,488.215
606,call,565.0,0.996972,27.715,0.034594,2026-12-18,0.907264,488.215
607,call,570.0,0.996972,25.995,0.034594,2026-12-18,0.890686,488.215
608,call,575.0,0.996972,25.040,0.034594,2026-12-18,0.883329,488.215


In [9]:
import numpy as np
surface_data["F"] = spot * np.exp((surface_data["rate"] - div) * surface_data["T"])
surface_data["k"] = np.log(surface_data["K"] / surface_data["F"])
surface_data["w"] = surface_data["iv"] ** 2 * surface_data["T"]
surface_data[["K", "w", "k", "expiry" ,"F"]]
#print(f"Spot: {spot}")

,K,w,k,expiry,F
32,490.0,0.000004,0.003657,2025-12-19,488.211519
61,400.0,0.022843,-0.185824,2025-12-26,481.683917
62,405.0,0.022367,-0.173401,2025-12-26,481.683917
63,410.0,0.017296,-0.161131,2025-12-26,481.683917
64,415.0,0.016003,-0.149010,2025-12-26,481.683917
...,...,...,...,...,...
605,560.0,0.838521,0.850421,2026-12-18,239.251664
606,565.0,0.820637,0.859310,2026-12-18,239.251664
607,570.0,0.790919,0.868120,2026-12-18,239.251664
608,575.0,0.777907,0.876854,2026-12-18,239.251664


In [10]:
theta = (surface_data.loc[surface_data["k"].abs().groupby(surface_data["T"]).idxmin()].set_index("T")["w"])
surface_data["theta"] = surface_data["T"].map(theta)
surface_data[["expiry", "T", "K", "w", "theta"]]


,expiry,T,K,w,theta
32,2025-12-19,0.000010,490.0,0.000004,0.000004
61,2025-12-26,0.018890,400.0,0.022843,0.001340
62,2025-12-26,0.018890,405.0,0.022367,0.001340
63,2025-12-26,0.018890,410.0,0.017296,0.001340
64,2025-12-26,0.018890,415.0,0.016003,0.001340
...,...,...,...,...,...
605,2026-12-18,0.996972,560.0,0.838521,3.227223
606,2026-12-18,0.996972,565.0,0.820637,3.227223
607,2026-12-18,0.996972,570.0,0.790919,3.227223
608,2026-12-18,0.996972,575.0,0.777907,3.227223


In [11]:
vega = IVEngine._vega(surface_data["iv"], surface_data["K"], surface_data["T"], surface_data["rate"], div, surface_data["F"])
surface_data["vega"] = np.clip(vega, 1e-6, None)
print(f"Spot price: ${spot:.2f}")
surface_data[["expiry", "K", "k", "vega"]]

Spot price: $488.21


,expiry,K,k,vega
32,2025-12-19,490.0,0.003657,0.098352
61,2025-12-26,400.0,-0.185824,12.433113
62,2025-12-26,405.0,-0.173401,13.532187
63,2025-12-26,410.0,-0.161131,12.849321
64,2025-12-26,415.0,-0.149010,13.678652
...,...,...,...,...
605,2026-12-18,560.0,0.850421,20.664184
606,2026-12-18,565.0,0.859310,19.812708
607,2026-12-18,570.0,0.868120,18.546201
608,2026-12-18,575.0,0.876854,17.850152


In [12]:
T_vals = surface_data["T"].values
T_unique = np.sort(surface_data["T"].unique())
T_to_index = {t: i for i, t in enumerate(T_unique)}
idx = np.array([T_to_index[t] for t in T_vals]) # index for each row to use on T_unique to find eta
x0 = np.r_[ -0.3, np.full(len(T_unique), 0.5) ]    # rho, eta_i-1



In [13]:
def ssvi_w(k, theta, phi, rho):
    w_ssvi = (
        1
        / 2
        * theta
        * (1 + rho * phi * k + np.sqrt((phi * k + rho) ** 2 + 1 - rho**2))
    )
    return w_ssvi


In [14]:
vegas = surface_data["vega"].values
weights = vegas / np.sum(vegas)

In [15]:
def objective(x, theta, k, w_mkt, idx):
    rho = x[0]
    eta = x[1:]
    phi = eta[idx] / np.sqrt(np.maximum(theta, 1e-12))
    w_model = ssvi_w(k, theta, phi, rho)
    error = (w_model - w_mkt)
    loss = np.dot(error * weights, error) 
    return loss

In [16]:
#def butterfly_constraint(x):
#    eta, rho = x
#    return 2 - eta * (1 + abs(rho))

def make_constraints(n_expiries):
    cons = []
    for i in range(n_expiries):
        cons.append({
            "type": "ineq",
            "fun": lambda x, i=i: 2 - x[1+i] * (1 + abs(x[0]))
        })
    return cons


In [17]:
from scipy.optimize import minimize

bounds = [(-1.0, 1.0)] + [(1e-5, 2)] * len(T_unique)
w_mkt = surface_data["w"].values
k = surface_data["k"].values
theta = surface_data["theta"].values


res = minimize(
    objective,
    x0=x0,
    args=(theta, k, w_mkt, idx),
    method="SLSQP",
    bounds=bounds,
    constraints=make_constraints(len(T_unique))
)

print(res.x)
print(f"Optimised Rho: {res.x[0]:.6f}")
print(f"Optimised Eta: {res.x[1:]}")

surface_data["w_ssvi"] = ssvi_w(k, theta, phi= res.x[1:][idx] / np.sqrt(np.maximum(theta, 1e-12)), rho=res.x[0]) #type: ignore
error = objective(res.x, theta, k, w_mkt, idx)
print(f"SSVI Loss: {error:.6f}")
w = surface_data["w"].values

err = np.abs(surface_data["w_ssvi"].values - w) # type: ignore
relative = err / np.maximum(w, 1e-6) # type: ignore
print(f"Mean Relative Error: {np.mean(relative)*100:.4f}%")


surface_data[["expiry", "K", "T", "w", "w_ssvi", "theta"]]



[-1.          0.5         0.50051264  0.50426135  0.51105372  0.51792745
  0.52728761  0.53803214  0.61544216  1.          1.          1.
  1.          1.          1.          1.          1.        ]
Optimised Rho: -1.000000
Optimised Eta: [0.5        0.50051264 0.50426135 0.51105372 0.51792745 0.52728761
 0.53803214 0.61544216 1.         1.         1.         1.
 1.         1.         1.         1.        ]
SSVI Loss: 0.039836
Mean Relative Error: 63.7115%


,expiry,K,T,w,w_ssvi,theta
32,2025-12-19,490.0,0.000010,0.000004,1.595145e-07,0.000004
61,2025-12-26,400.0,0.018890,0.022843,4.744820e-03,0.001340
62,2025-12-26,405.0,0.018890,0.022367,4.517210e-03,0.001340
63,2025-12-26,410.0,0.018890,0.017296,4.292393e-03,0.001340
64,2025-12-26,415.0,0.018890,0.016003,4.070300e-03,0.001340
...,...,...,...,...,...,...
605,2026-12-18,560.0,0.996972,0.838521,1.699487e+00,3.227223
606,2026-12-18,565.0,0.996972,0.820637,1.683519e+00,3.227223
607,2026-12-18,570.0,0.996972,0.790919,1.667691e+00,3.227223
608,2026-12-18,575.0,0.996972,0.777907,1.652001e+00,3.227223


In [18]:
err = surface_data["w_ssvi"].values - surface_data["w"].values
rmse  = np.sqrt(np.mean(err**2))
nrmse = rmse / np.mean(surface_data["w"].values)
print("NRMSE:", nrmse)


NRMSE: 0.7085778382968106
